## 1. Environment Setup
Check GPU availability and install necessary libraries.

In [ ]:
# Check GPU status
!nvidia-smi

In [ ]:
# Install required packages
print("Installing required packages...")
print("="*70)

# Install packages
%pip install -q ultralytics roboflow openvino nncf matplotlib seaborn pandas

print("\n" + "="*70)
print("✓ Packages installed successfully!")

In [ ]:
# Import necessary libraries
import os
import shutil
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ultralytics import YOLO
from google.colab import userdata
from roboflow import Roboflow

# Roboflow configuration
try:
    ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
    ROBOFLOW_WORKSPACE = userdata.get('ROBOFLOW_WORKSPACE')
    ROBOFLOW_PROJECT = userdata.get('ROBOFLOW_PROJECT')
    ROBOFLOW_VERSION = userdata.get('ROBOFLOW_VERSION')
    print("✓ Roboflow configuration loaded!")
except Exception as e:
    print(f"⚠ Error loading Roboflow config: {e}")
    print("Please ensure you have set up the secrets in Google Colab.")

# Model configuration
MODEL_NAME = "yolo11n-seg-version-1-0-0"
MODEL_PATH = f"/content/{MODEL_NAME}.pt"

print(f"Model: {MODEL_NAME}")

## 2. Download Dataset
Download the dataset from Roboflow to be used for INT8 calibration and model evaluation.

In [ ]:
# Download dataset
print("Downloading dataset from Roboflow...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(int(ROBOFLOW_VERSION))
dataset = version.download("yolov11")

DATASET_PATH = f"{dataset.location}/data.yaml"
print(f"\n✓ Dataset downloaded to: {dataset.location}")
print(f"✓ Data YAML: {DATASET_PATH}")

## 3. Export Models to OpenVINO
We will export the PyTorch model to two OpenVINO formats:
1.  **FP16 (Half-Precision):** Good balance of speed and accuracy.
2.  **INT8 (8-bit Quantization):** Maximum speedup on CPU/iGPU, requires calibration data.

In [ ]:
# Load the YOLO11n-seg PyTorch model
print(f"Loading PyTorch model from: {MODEL_PATH}")
model = YOLO(MODEL_PATH)
print("✓ Model loaded successfully!")

# 1. Export to OpenVINO FP16
print("\n" + "="*70)
print("EXPORTING: OpenVINO FP16")
print("="*70)
fp16_export_path = model.export(
    format="openvino",
    imgsz=640,
    half=True,  # Enable FP16 quantization
)
print(f"✓ OpenVINO FP16 model saved to: {fp16_export_path}")

# 2. Export to OpenVINO INT8
print("\n" + "="*70)
print("EXPORTING: OpenVINO INT8")
print("="*70)
print("Starting INT8 export with calibration (this may take a while)...")
int8_export_path = model.export(
    format="openvino",
    imgsz=640,
    int8=True,          # Enable INT8 quantization
    data=DATASET_PATH,  # Dataset for calibration
    fraction=0.5,       # Use 50% of data for calibration
)
print(f"✓ OpenVINO INT8 model saved to: {int8_export_path}")

## 4. Comprehensive Model Evaluation
Evaluate all three models (PyTorch, FP16, INT8) on the **Test Dataset** to compare their performance.

In [ ]:
# Define model paths
models_to_eval = {
    'PyTorch': MODEL_PATH,
    'OpenVINO_FP16': fp16_export_path,
    'OpenVINO_INT8': int8_export_path
}

print("=" * 70)
print("STARTING COMPREHENSIVE EVALUATION ON TEST DATASET")
print("=" * 70)

all_metrics = {}

for name, path in models_to_eval.items():
    print(f"\nEvaluating: {name}...")
    print(f"Path: {path}")
    print("-" * 30)

    # Load model
    eval_model = YOLO(path)

    # Run validation on Test set
    # Note: For OpenVINO models, we might need to ensure the task is specified if not auto-detected,
    # but YOLO class usually handles it.
    metrics = eval_model.val(data=DATASET_PATH, split='test', verbose=False)

    # Store metrics
    all_metrics[name] = {
        # Box Metrics
        'Box_mAP50-95': metrics.box.map,
        'Box_mAP50': metrics.box.map50,
        'Box_mAP75': metrics.box.map75,
        'Box_Precision': metrics.box.mp,
        'Box_Recall': metrics.box.mr,
        # Mask Metrics
        'Mask_mAP50-95': metrics.seg.map,
        'Mask_mAP50': metrics.seg.map50,
        'Mask_mAP75': metrics.seg.map75,
        'Mask_Precision': metrics.seg.mp,
        'Mask_Recall': metrics.seg.mr,
    }

    print(f"  ✓ Box mAP50-95:  {metrics.box.map:.4f}")
    print(f"  ✓ Mask mAP50-95: {metrics.seg.map:.4f}")

print("\n" + "="*70)
print("✓ Evaluation completed for all models!")

## 5. Visualization & Comparison
Create high-quality charts to visualize the performance differences between the models.

In [ ]:
# Prepare data for plotting
metrics_df = pd.DataFrame(all_metrics).T
print("Metrics Summary:")
display(metrics_df)

# Configuration for plot
Y_AXIS_MIN = 0.9
Y_AXIS_MAX = 1.02
# Custom colors for better contrast (Brighter/Nicer): PyTorch (Sky Blue), FP16 (Emerald Green), INT8 (Coral Red)
MODEL_COLORS = ['#3498db', '#2ecc71', '#E97F4A']

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')

# Create figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle('YOLOv11 Segmentation Model Comparison: PyTorch vs OpenVINO (FP16 & INT8)', fontsize=18, fontweight='bold', y=1.05)

# Metrics to plot
plot_metrics = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
x = np.arange(len(plot_metrics))
width = 0.25  # Width of bars

# --- Subplot 1: Box Metrics ---
ax1 = axes[0]
for i, model_name in enumerate(models_to_eval.keys()):
    values = [all_metrics[model_name][f'Box_{m}'] for m in plot_metrics]
    offset = (i - 1) * width
    # Use custom color if available, else default
    color = MODEL_COLORS[i] if i < len(MODEL_COLORS) else None
    bars = ax1.bar(x + offset, values, width, label=model_name, color=color, edgecolor='black', linewidth=0.5)

    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax1.annotate(f'{height:.3f}',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points",
                     ha='center', va='bottom', fontsize=9, rotation=90)

ax1.set_ylabel('Score', fontsize=12)
ax1.set_title('Bounding Box Metrics', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(plot_metrics, fontsize=11)
ax1.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)
# Move legend to top
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, framealpha=0.9)
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# --- Subplot 2: Mask Metrics ---
ax2 = axes[1]
for i, model_name in enumerate(models_to_eval.keys()):
    values = [all_metrics[model_name][f'Mask_{m}'] for m in plot_metrics]
    offset = (i - 1) * width
    # Use custom color if available, else default
    color = MODEL_COLORS[i] if i < len(MODEL_COLORS) else None
    bars = ax2.bar(x + offset, values, width, label=model_name, color=color, edgecolor='black', linewidth=0.5)

    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax2.annotate(f'{height:.3f}',
                     xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points",
                     ha='center', va='bottom', fontsize=9, rotation=90)

ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Segmentation Mask Metrics', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(plot_metrics, fontsize=11)
ax2.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)
# Move legend to top
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, framealpha=0.9)
ax2.grid(axis='y', linestyle='--', alpha=0.7)

# Save chart
chart_filename = "model_comparison_chart.png"
plt.tight_layout()
plt.savefig(chart_filename, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Comparison chart saved to: {chart_filename}")

## 6. Save Results to Google Drive
Archive all artifacts: models, reports, and charts.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_base_dir = f"/content/drive/MyDrive/YOLO_Models/{MODEL_NAME}"
output_dir = f"{output_base_dir}/Summary_{timestamp}"

os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")
print("="*70)

# 1. Save Models
print("\n1. Saving Models...")
# PyTorch
shutil.copy2(MODEL_PATH, os.path.join(output_dir, f"{MODEL_NAME}.pt"))
print(f"   ✓ Saved PyTorch model")

# OpenVINO FP16
dest_fp16 = os.path.join(output_dir, "openvino_fp16")
if os.path.exists(fp16_export_path):
    shutil.copytree(fp16_export_path, dest_fp16)
    print(f"   ✓ Saved OpenVINO FP16 model")

# OpenVINO INT8
dest_int8 = os.path.join(output_dir, "openvino_int8")
if os.path.exists(int8_export_path):
    shutil.copytree(int8_export_path, dest_int8)
    print(f"   ✓ Saved OpenVINO INT8 model")

# 2. Save Chart
print("\n2. Saving Charts...")
shutil.copy2(chart_filename, os.path.join(output_dir, chart_filename))
print(f"   ✓ Saved comparison chart")

# 3. Generate and Save Report
print("\n3. Saving Evaluation Report...")
report_file = os.path.join(output_dir, "evaluation_report.txt")

with open(report_file, 'w') as f:
    f.write("="*70 + "\n")
    f.write(f"YOLOv11 Segmentation Model Evaluation Report\n")
    f.write(f"Model: {MODEL_NAME}\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*70 + "\n\n")

    f.write("1. MODEL CONFIGURATIONS\n")
    f.write("-" * 30 + "\n")
    f.write(f"PyTorch:       Original FP32\n")
    f.write(f"OpenVINO FP16: Half-precision floating point\n")
    f.write(f"OpenVINO INT8: 8-bit Integer Quantization (Calibrated on {DATASET_PATH})\n\n")

    f.write("2. METRICS COMPARISON (TEST DATASET)\n")
    f.write("-" * 30 + "\n")

    # Write table header
    headers = ["Metric", "PyTorch", "FP16", "INT8", "Diff(INT8-PT)"]
    f.write(f"{headers[0]:<20} {headers[1]:<12} {headers[2]:<12} {headers[3]:<12} {headers[4]:<15}\n")
    f.write("-" * 75 + "\n")

    # Write metrics
    for metric_key in all_metrics['PyTorch'].keys():
        pt_val = all_metrics['PyTorch'][metric_key]
        fp16_val = all_metrics['OpenVINO_FP16'][metric_key]
        int8_val = all_metrics['OpenVINO_INT8'][metric_key]
        diff = int8_val - pt_val

        f.write(f"{metric_key:<20} {pt_val:<12.4f} {fp16_val:<12.4f} {int8_val:<12.4f} {diff:<+15.4f}\n")

    f.write("\n")
    f.write("3. DEGRADATION ANALYSIS (INT8 vs PyTorch)\n")
    f.write("-" * 30 + "\n")
    box_deg = (all_metrics['OpenVINO_INT8']['Box_mAP50-95'] - all_metrics['PyTorch']['Box_mAP50-95']) / all_metrics['PyTorch']['Box_mAP50-95'] * 100
    mask_deg = (all_metrics['OpenVINO_INT8']['Mask_mAP50-95'] - all_metrics['PyTorch']['Mask_mAP50-95']) / all_metrics['PyTorch']['Mask_mAP50-95'] * 100

    f.write(f"Box mAP50-95 Change:  {box_deg:+.2f}%\n")
    f.write(f"Mask mAP50-95 Change: {mask_deg:+.2f}%\n")

print(f"   ✓ Saved evaluation report")

print("\n" + "="*70)
print("✓ ALL TASKS COMPLETED SUCCESSFULLY")
print(f"Results saved to: {output_dir}")